# Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы.

In [145]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [146]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [147]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(min_df=0.01, max_df=0.95,lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=0.01,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'put', 'thence', 'except', 'him', 'thereby', 'forty', 'go', 'meanwhile', 'whereupon', 'wherever', 'it', 'mill', 'than', 'least', 'eight', 'over', 'anyhow', 'sometime', 'along', 'because', 'five', 'behind', 'without', 'front', 'whether', 'why', 'any', 'noone', 'ever', 'eleven', ...ong', 'them', 'less', 'thin', 'all', 'had', 'con', 'take', 'eg', 'mostly', 'un', 'else', 'already'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [148]:
len(vectorizer.vocabulary_)

1142

In [157]:
X_train = vectorizer.transform(newsgroups_train.data)
X_train.shape

(11314, 1142)

предварительная работа проведена, теперь алгоритм:

In [150]:
a=X_train.nonzero()
a=np.transpose(a)
z=np.zeros(X_train.shape,dtype = np.int8)
nkw=np.zeros((20,X_train.shape[1]))
ndk=np.zeros((20,X_train.shape[0]))
nk=np.zeros(20)

In [151]:
for i in a :
    c=np.random.randint(20)
    z[i[0],i[1]]=c
    nkw[c,i[1]]+=1
    ndk[c,i[0]]+=1
    nk[c]+=1

In [152]:
beta=50/X_train.shape[1]
def LDA(X_train,nkw,ndk,nk,z) :
    p=np.zeros(20)
    for j in range(1000) :
        for i in a:
            ndk[z[i[0],i[1]],i[0]]-=1
            nkw[z[i[0],i[1]],i[1]]-=1
            nk[z[i[0],i[1]]]-=1
            for k in range(20) :
                p[k]=(ndk[k,i[0]]+1/20)*(nkw[k,i[1]]+beta)/(nk[k]+50)
            p=p.cumsum()
            v=np.random.rand()
            v=v*p[-1]
            for k in range(20) :
                if p[k] > v :
                    z[i[0],i[1]]=k
                    break
            ndk[z[i[0],i[1]],i[0]]+=1
            nkw[z[i[0],i[1]],i[1]]+=1
            nk[z[i[0],i[1]]]+=1
    return z,ndk,nkw

In [153]:
z,ndk,nkw=LDA(X_train,nkw,ndk,nk,z)

полечаем теги с такими топ 10:

In [170]:
for i in range(20):
  mask = (nkw[i] >= np.sort(nkw[i])[-10])
  print("%d Topic: {" % (i + 1), " ".join(vectorizer.inverse_transform(mask)[0]),"}")

1 Topic: { don just know like ll people really say think want }
2 Topic: { don government gun just law people right rights state think }
3 Topic: { believe does don just know like people point say think }
4 Topic: { file like problem running set thanks use using window windows }
5 Topic: { april government national new people states today world year years }
6 Topic: { ca com cs edu email fax internet mail phone university }
7 Topic: { don good just know like really think time ve way }
8 Topic: { chip clipper data encryption government just key keys phone public secure }
9 Topic: { available free including information list note number provide send use }
10 Topic: { 10 11 12 13 14 15 16 17 20 25 }
11 Topic: { does drive hard just know like problem thanks use work }
12 Topic: { does file files ftp know like program thanks use version windows }
13 Topic: { advance appreciated does hi information know like mail post thanks }
14 Topic: { bible christ christian christians does god jesus life 

Ещё раз выведем названия топиков ,которые были даны изначально: 

In [173]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Попробуем их соотнести с нашими топиками после применения алгоритма:

    1)Напримар тег  'soc.religion.christian' хорошо соотносится с Топиком №14:
            { bible christ christian christians does god jesus life people say }
    2)тег 'talk.politics.guns' хорошо соотносится с Топиком №2:
        { don government gun just law people right rights state think }
    3)тег 'talk.politics.mideast'   хорошо соотносится с Топиком №5:
        { april government national new people states today world year years }  
    4)тег 'rec.sport.hockey'   хорошо соотносится с Топиком №17:
        { game games good like play players season team think year }
    5)тег 'misc.forsale'   хорошо соотносится с Топиком №19:
        { best condition edu interested new offer price sale sell shipping }
    6)тег 'comp.windows.x'   хорошо соотносится с Топиком №4
    7)тег 'comp.sys.ibm.pc.hardware'   хорошо соотносится с Топиком №6
    8)тег 'sci.electronics'   хорошо соотносится с Топиком №8
    9)тег 'comp.graphics'   хорошо соотносится с Топиком №16
    И ещё несколько соотношений
Исходя из этих наблюдений, делаем вывод что алгоритм восстанавлиает почти такое же разбиение на темы, но всё же не полностью, а лишь частично.